In [9]:
import os, json
import pandas as pd
import numpy as np
import math
import datetime as dt
import requests
from concurrent import futures
# import asyncio
# import aiohttp
# from aiohttp import ClientSession, client_exceptions
# from asyncio import Semaphore, ensure_future, gather, run
pd.set_option("display.max_colwidth", 10000)

In [2]:
#url list builder function
def url_builder(lon1, lat1, lon2, lat2):
    """
    Pre-assemble the list of URLs to pass to the routing machine to cut down on compute time
    """
    url = ('http://localhost:5000/route/v1/driving/{},{};{},{}?overview=false'
           .format(lon1, lat1, lon2, lat2))
    return url


In [3]:
#read in data
df = pd.read_csv("C:/Users/Mark Hansen/daily_data/grouped/bus_points_2-17.csv")

In [4]:
#convert timestamps to datetime format
df['call_time'] = pd.to_datetime(df['call_time'])
#create report time variable (difference between the time the bus reported its coordinates and my ping)
df['report_time'] = df['call_time'] - pd.to_timedelta(df['seconds_since_report'], unit='s')

In [5]:
#sort by bus_id and report_time, calculate time deltas
df = df.sort_values(by=['bus_id', 'report_time'])
df['diff'] = df.groupby('bus_id')['report_time'].diff()

In [6]:
#if the ping response has the same report as before, the difference between report times is 0, thus useless
df = df[df['diff'].dt.total_seconds() != 0]

In [7]:
#group and add prior report's lat long
dfg = df.groupby('bus_id')
df ['p_latitude'] = dfg['latitude'].shift(1)
df ['p_longitude'] = dfg['longitude'].shift(1)
#i think the NAs are puttings 0,0 coordinates into my distance calculations?
df = df.dropna(subset=['diff'])


In [8]:
#build url list

url_list = df[['p_latitude', 'p_longitude', 'latitude','longitude']].apply(
    lambda x: url_builder(x[1], x[0], x[3], x[2]), axis=1)

In [ ]:
# #multithreading
# responses = []
# def get_resp(url):
#     resp = requests.get(url)
#     data = resp.json()
#     return data['routes'][0]['distance']
# with futures.ThreadPoolExecutor(max_workers = 2000) as executor:
#     resps = executor.map(get_resp, url_list)
# responses = list(resps)

In [ ]:
#async option 1

# distances = []
# async def fetch(url, session):
#     async with session.get(url) as response:
#         return await response.json()
    
# async def bound_fetch(sem, url, session):
#     async with sem:
#         await fetch(url, session)
        
# async def run(session, urls):
#     tasks = []
#     sem = asyncio.Semaphore(2000)
#     for url in urls:
#         task = asyncio.ensure_future(bound_fetch(sem, url, session))
#         tasks.append(task)
#     responses = asyncio.gather(*tasks)
#     await responses
# async with ClientSession() as session:
#    await run(session, url_list)
    

In [ ]:
#async option 2
# limit = 500
# distances = []
# async def scrape(url_list):
#     tasks = list()
#     sem = Semaphore(limit)
    
#     async with ClientSession() as session:
#         for url in url_list:
#             task = ensure_future(scrape_bounded(url, sem, session))
#             tasks.append(task)
#         result = await gather(*tasks)
#     return result

# async def scrape_bounded(url, sem, session):
#     async with sem:
#         return await scrape_one(url, session)

# async def scrape_one(url, session):
#     async with session.get(url) as response:
#         content = await response.json()
#     content = content['routes'][0]['distance']
    
#     return content


# loop = asyncio.get_event_loop()
# distances = loop.create_task(scrape(url_list))

        

In [ ]:

# df['distance'] = df[['url']].apply(
# lambda x: make_request(x[0]), axis=1)

In [ ]:
#get hours
# df['diff'] = df['diff'].dt.total_seconds()
# df['diff'] = df['diff']/3600

In [ ]:
#convert KM to miles
# df['distance'] = df['distance']/1.609

In [ ]:
#speed is distance over time
# df['mph'] = df['distance']/df['diff']

In [ ]:
#df.to_csv('C:/Users/Mark Hansen/daily_data/grouped/bus_points_speed2.csv')